In [33]:
# Imports
import talib
import pandas as pd
import yfinance as yf
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [34]:
# Download historical market data
ticker = "SPY"
stock = yf.Ticker(ticker)

# get historical market data
hist = stock.history(period="3y")

In [35]:
# Download historical data as dataframe
spy_data = yf.download('SPY', start='2020-01-01', end='2023-06-30')

[*********************100%***********************]  1 of 1 completed


In [36]:
spy_data.to_csv('SPY_data.csv')

In [37]:
spy_df =pd.read_csv('SPY_data.csv',index_col=0,parse_dates=True)
spy_df.dropna()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,323.540009,324.890015,322.529999,324.869995,307.375702,59151200
2020-01-03,321.160004,323.640015,321.100006,322.410004,305.048157,77709700
2020-01-06,320.489990,323.730011,320.359985,323.640015,306.211853,55653900
2020-01-07,323.019989,323.540009,322.239990,322.730011,305.350861,40496400
2020-01-08,322.940002,325.779999,322.670013,324.450012,306.978394,68296000
...,...,...,...,...,...,...
2023-06-23,432.929993,435.059998,432.470001,433.209991,433.209991,92074500
2023-06-26,432.619995,434.609985,431.190002,431.440002,431.440002,72823600
2023-06-27,432.350006,436.809998,431.880005,436.170013,436.170013,72813700


In [38]:
# Assuming you have a pandas DataFrame named 'df' containing your data
ema_period = 14
spy_df['EMA'] = talib.EMA(spy_df['Close'], timeperiod=ema_period)

In [39]:
spy_df.dropna(how='any', inplace=True) 

In [46]:
spy_df['Signal'] = 0  # Initialize all signals as 0
spy_df.loc[spy_df['Close'] > spy_df['EMA'], 'Signal'] = 1  # Assign buy signal
spy_df.loc[spy_df['Close'] < spy_df['EMA'], 'Signal'] = -1  # Assign sell signal
spy_df.head()

,Open,High,Low,Close,Adj Close,Volume,EMA,Signal,Entry/Exit
Date,,,,,,,,,
2020-01-22,332.239990,332.950012,331.170013,331.339996,313.497314,48914900,327.111433,1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-23,330.630005,332.170013,329.410004,331.720001,313.856781,51963000,327.725909,1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-24,332.440002,332.529999,327.359985,328.769989,311.065643,87578400,327.865119,1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-27,323.029999,325.119995,322.660004,323.500000,306.079468,84062500,327.283103,-1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-28,325.059998,327.850006,323.600006,326.890015,309.286865,63834000,327.230692,-1,<bound method Series.diff of Date\n2020-01-22 ...


In [44]:
spy_df['Entry/Exit'] = spy_df['Signal'].diff
spy_df.head()

,Open,High,Low,Close,Adj Close,Volume,EMA,Signal,Entry/Exit
Date,,,,,,,,,
2020-01-22,332.239990,332.950012,331.170013,331.339996,313.497314,48914900,327.111433,1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-23,330.630005,332.170013,329.410004,331.720001,313.856781,51963000,327.725909,1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-24,332.440002,332.529999,327.359985,328.769989,311.065643,87578400,327.865119,1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-27,323.029999,325.119995,322.660004,323.500000,306.079468,84062500,327.283103,-1,<bound method Series.diff of Date\n2020-01-22 ...
2020-01-28,325.059998,327.850006,323.600006,326.890015,309.286865,63834000,327.230692,-1,<bound method Series.diff of Date\n2020-01-22 ...


In [45]:
X = spy_df['EMA'].values.reshape(-1, 1)  # Reshape to a 2D array if needed
y = spy_df['Signal'].values


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [18]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred =  svm_model.predict(X_train_scaled)

# Review the model's predicted values
svm_pred[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [24]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_train, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       249
           1       0.64      1.00      0.78       443

    accuracy                           0.64       692
   macro avg       0.32      0.50      0.39       692
weighted avg       0.41      0.64      0.50       692



In [25]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)
print(len(testing_signal_predictions))
buy_item=0
sell_item = 0
for item in testing_signal_predictions:
    if item == 1:
        buy_item +=1
    else:
        sell_item +=1
print(buy_item)
print(sell_item)
    

174
174
0


In [26]:
# Evaluate the model's ability to predict the trading signal for the testing data
svm_testing_report = classification_report(y_test, testing_signal_predictions)
print(svm_testing_report)

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        65
           1       0.63      1.00      0.77       109

    accuracy                           0.63       174
   macro avg       0.31      0.50      0.39       174
weighted avg       0.39      0.63      0.48       174

